In [1]:
from tensorflow.keras import models
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Input, Conv2D, Permute, Average
from tensorflow.keras.layers import MaxPooling2D, ZeroPadding2D, Cropping2D, UpSampling2D, AveragePooling2D
from tensorflow.keras.layers import Flatten, Dropout, Reshape, ZeroPadding1D, Cropping1D, TimeDistributed
from tensorflow.keras.layers import GlobalAveragePooling1D, AveragePooling1D, UpSampling1D, MaxPooling1D, AveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K

# Other useful packages:
from plotly.subplots import make_subplots
import numpy as np
from plotly import graph_objs as go
from tqdm.notebook import trange
import pandas as pd
from scipy.interpolate import interp1d
from IPython.display import clear_output
from scipy.optimize import curve_fit

In [2]:
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        'title': 'Time (\u03BCs)',
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 2.5,
        'tickwidth': 2.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'title': 'Coherence',
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 2.5,
        'tickwidth': 2.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 22,
            },
    'colorway': ["#d9ed92","#b5e48c","#99d98c","#76c893","#52b69a","#34a0a4","#168aad","#1a759f","#1e6091","#184e77"]
}

## Training data pre-processsing functions

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

# %%
# For data interpolation
def interpData(x,y,xNew):
	f_interp = interp1d(x,y)
	yNew = f_interp(xNew)
	return yNew


# %%
# Following functions are required to test the accuracy of the predictions
# Create CPMG-like pulse timing array

def cpmgFilter(n, Tmax):
    tpi = np.empty([n])
    for i in range(n):
        tpi[i]= Tmax*(((i+1)-0.5)/n)
    return tpi


# %%
# Generate filter function for a given pulse sequence

def getFilter(n,w0,piLength,Tmax):
    tpi = cpmgFilter(n,Tmax)
    f = 0
    for i in range(n):
        f = ((-1)**(i+1))*(np.exp(1j*w0*tpi[i]))*np.cos((w0*piLength)/2) + f

    fFunc = (1/2)*(( np.abs(1+((-1)**(n+1))*np.exp(1j*w0*Tmax)+2*f) )**2)/(w0**2)
    return fFunc


# %%
# Generate decoherence curve corresponding to a noise spectrum (input shape = variable1.size x w.size)

def getCoherence(S,w0,T0,n,piLength):
    steps = T0.size
    C_invert = np.empty([S.shape[0],steps,])
    for i in range(steps):
        integ = getFilter(n,np.squeeze(w0),piLength,T0[i])*S/np.pi
        integ_ans = np.trapz(y=integ,x=np.squeeze(w0))
        C_invert[:,i] = np.exp(integ_ans)
    return C_invert

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ls "/content/gdrive/My Drive/Sid-Research/IITM/Research/ML for fluid dynamics/RS_AK_code/"

 Data_SD.npz
'History1_580.95_fil=16_ker=30_dr0.03_ps=2_LRini=0.001_LRmin=1e-06_bs=64_ep=100_NOISE_TYPES=lor.csv'
'History2_580.95_fil=16_ker=30_dr0.03_ps=2_LRini=0.001_LRmin=1e-06_bs=64_ep=100_NOISE_TYPES=lor.csv'
'MODEL_580.95_fil=16_ker=30_dr0.03_ps=2_LRini=0.001_LRmin=1e-06_bs=64_ep=100_NOISE_TYPES=lor'
 __pycache__
 solvers.py
 TrainingData.ipynb
 TrainingData_SD.ipynb


In [5]:
# !ls "/content/gdrive/My Drive/Research/ML for fluid dynamics/Data"

In [6]:
path = F"/content/gdrive/My Drive/Sid-Research/IITM/Research/ML for fluid dynamics/RS_AK_code/"

## Model creation function

In [ ]:
from tensorflow.keras import layers

def get_model( filter_nb, kernel_size, pool_size, dropout_rate, xtrain_size, ytrain_size=501 ):
	model = models.Sequential()
	model.add( layers.Input( shape=(xtrain_size, 1) ) )
	model.add( layers.Conv1D(filter_nb,kernel_size,activation="relu", padding='same' ) )
	model.add( layers.Conv1D(filter_nb,kernel_size,activation="relu", padding='same' ) )
	model.add( layers.MaxPooling1D( pool_size=pool_size, padding="same") )
	model.add( layers.Conv1D(filter_nb//2, kernel_size,activation="relu", padding='same' ) )
	model.add( layers.MaxPooling1D( pool_size=pool_size, padding="same") )
	model.add( layers.Conv1D(filter_nb//4,kernel_size,activation="relu", padding='same' ) )
	model.add( layers.MaxPooling1D( pool_size=pool_size, padding="same") )

	model.add( layers.Conv1D( filter_nb//4,kernel_size,activation="relu", padding='same' ) )
	model.add( layers.UpSampling1D( size=pool_size ) )
	model.add( layers.Conv1D( filter_nb//2,kernel_size,activation="relu", padding='same' ) )
	model.add( layers.UpSampling1D( size=pool_size ) )

	model.add( layers.Conv1D( filter_nb,kernel_size,activation="relu", padding='same' ) )
	model.add( layers.UpSampling1D( size=pool_size ) )

	model.add( layers.Conv1D( filter_nb,kernel_size,activation="relu", padding='same' ) )
	model.add( layers.Conv1D( 1, kernel_size, activation="relu", padding='same' ) )
	model.add( layers.Flatten() )
	model.add( layers.Dropout( dropout_rate ) )
	model.add( layers.Dense(ytrain_size, activation='linear') )

	return model

In [7]:
k_size = (8,8)
filter_num = 64

model2d = Sequential()
model2d.add(Input(shape=(15,15,1)))
model2d.add(ZeroPadding2D(((0,1),(0,1))))
model2d.add(Conv2D(filters=filter_num,kernel_size=k_size, activation="relu",padding='same'))
model2d.add(MaxPooling2D((2,2),padding="same"))
model2d.add(Conv2D(filters=filter_num/2,kernel_size=k_size,activation="relu",padding='same'))
# model2d.add(MaxPooling2D((2,2),padding="same"))
model2d.add(Conv2D(filters=filter_num/4,kernel_size=k_size,activation="relu",padding='same'))
# model2d.add(UpSampling2D((2,2)))
model2d.add(Conv2D(filters=filter_num/2,kernel_size=k_size,activation="relu",padding='same'))
model2d.add(UpSampling2D((2,2)))
model2d.add(Conv2D(filters=filter_num,kernel_size=k_size,activation="relu",padding='same'))
model2d.add(Conv2D(filters=1,kernel_size=k_size,activation="relu",padding='same'))
model2d.add(Dropout(0.05))
model2d.add(Cropping2D(cropping=((0,1),(0,1))))
model2d.add(Reshape((15,15)))

## Training the model

In [8]:
from sklearn.model_selection import train_test_split

#=============================================
#== IMPORTING DATA
#=============================================

#== import the data
# data_1 = np.load(path+"/Lor_2.npz")
# data_1 = np.load(path+"/Data.npz")
data_1 = np.load(path+"Data_SD.npz")

print('-- data loaded')
print(data_1.files)

#== format the data for the training stage

# x_train, x_test, y_train, y_test = train_test_split(data_1['network_in'], data_1['network_out'], test_size=0.15)
x_train, x_test, y_train, y_test = train_test_split(data_1['vTrain_2d'], data_1['fTrain_2d'], test_size=0.15)

print('-- data split for training:')
print("  x_train = ",np.shape(x_train))
print("  y_train = ",np.shape(y_train))
print("  x_test = ",np.shape(x_test))
print("  y_test = ",np.shape(y_test))

-- data loaded
['vTrain', 'fTrain', 'vTrain_2d', 'fTrain_2d']
-- data split for training:
  x_train =  (17000, 15, 15)
  y_train =  (17000, 15, 15)
  x_test =  (3000, 15, 15)
  y_test =  (3000, 15, 15)


In [ ]:
c_check = getCoherence(data_1['network_out'],data_1['omega'],data_1['time'],1,100e-9)
n_plot = 10
rand_set = np.random.randint(0,data_1['network_in'].shape[0],(n_plot,))
seq_colors = ["#d9ed92","#b5e48c","#99d98c","#76c893","#52b69a","#34a0a4","#168aad","#1a759f","#1e6091","#184e77"]
fig = go.Figure()
for i in range(n_plot):
  fig.add_scatter(x=data_1['time']*1e6,y=data_1['network_in'][rand_set[i]],line=dict(width=2,color=seq_colors[i]),opacity=0.8,name=f'{rand_set[i]}')
  fig.add_scatter(x=data_1['time']*1e6,y=c_check[rand_set[i]],line=dict(width=3,color=seq_colors[i]),opacity=1,name=f'{rand_set[i]}')

fig.update_layout(template=fig_template)
fig.layout.xaxis.title = 'Total time (\N{greek small letter mu}s)'
fig.layout.yaxis.title = 'Coherence'
fig.update_xaxes(range=[0,400])
fig

In [ ]:
#==================================================
#== create the neural net for noise spectroscopy ==
#==================================================

FILTER_NB=20
KERNEL_SIZE=20
DROPOUT_RATE=0.05
POOL_SIZE=2
X_TRAIN_SIZE = np.shape(x_train)[-1]

model = get_model( filter_nb=FILTER_NB, kernel_size=KERNEL_SIZE, pool_size=POOL_SIZE,\
                  dropout_rate=DROPOUT_RATE, xtrain_size=X_TRAIN_SIZE )
print('-- model created')

-- model created


In [ ]:
#==============================================
#== create the neural net for fluid dynamics ==
#==============================================

FILTER_NB=16
KERNEL_SIZE=30
DROPOUT_RATE=0.03
POOL_SIZE=2
X_TRAIN_SIZE = 225
Y_TRAIN_SIZE = 225

model = get_model( filter_nb=FILTER_NB, kernel_size=KERNEL_SIZE, pool_size=POOL_SIZE,\
                  dropout_rate=DROPOUT_RATE, xtrain_size=X_TRAIN_SIZE, ytrain_size=Y_TRAIN_SIZE )
print('-- model created')

-- model created


In [9]:
model2d.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPaddin  (None, 16, 16, 1)         0         
 g2D)                                                            
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 64)        4160      
                                                                 
 max_pooling2d (MaxPooling2  (None, 8, 8, 64)          0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 32)          131104    
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 16)          32784     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 32)          3

In [ ]:
from tensorflow.keras import optimizers, callbacks
import time

#=============================================
#== training the neural net
#=============================================

#-- set up the hyperparameters
BATCH_SIZE=64
EPOCHS=200
INIT_LR=1e-3
MIN_LR=1e-6
RED_FACTOR=0.5
DROPOUT_RATE=0.03

reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss",factor=RED_FACTOR,patience=10,verbose=True,\
    mode="auto",min_delta=0.001,cooldown=0,min_lr=MIN_LR)  #-- define LR reduction strategy
opt = optimizers.Adam(learning_rate=INIT_LR)  #-- define optimizer
model2d.compile(loss='MAPE', optimizer=opt)  #-- compilation

#-- fit the model
print('-- beginning fit')
t1 = time.time()
history_ = model2d.fit( x_train, y_train, BATCH_SIZE, epochs=EPOCHS,\
                        validation_data=(x_test, y_test), verbose=True, callbacks=[reduce_lr])
final_accuracy = np.round(history_.history['val_loss'][-1],2)
print('-- fit complete')
print('-- time taken=', time.time() - t1)
print('-- final accuracy=', final_accuracy)

#-- define useful filename
filename=str( np.round(history_.history['val_loss'][-1],2) )\
			+"_fil="+str(FILTER_NB)+"_ker="+str(KERNEL_SIZE)+"_dr"+str(DROPOUT_RATE)\
            +"_ps="+str(POOL_SIZE)+'_LRini='+str(1e-3)+'_LRmin='+str(1e-6)\
            +'_bs='+str(BATCH_SIZE)+"_ep="+str(EPOCHS)+"_NOISE_TYPES=lor"
print("-- filename = "+filename)

#-- save the model in a folder so the model can be easily
#-- loaded and reused later
model2d.save(path+'/MODEL2d_'+filename, overwrite=True)
history_stored = pd.read_csv(path+'/History_'+ filename +'.csv')

-- beginning fit
Epoch 1/200
266/266 [==============================] - 71s 262ms/step - loss: 116.9539 - val_loss: 91.2192 - lr: 0.0010
Epoch 2/200
266/266 [==============================] - 68s 255ms/step - loss: 91.2729 - val_loss: 91.2192 - lr: 0.0010
Epoch 3/200
266/266 [==============================] - 69s 258ms/step - loss: 91.2730 - val_loss: 91.2192 - lr: 0.0010
Epoch 4/200
266/266 [==============================] - 68s 255ms/step - loss: 91.2730 - val_loss: 91.2192 - lr: 0.0010
Epoch 5/200
210/266 [======================>.......] - ETA: 13s - loss: 91.2130

In [ ]:
# pd.DataFrame(history_.history).to_csv(path+'/History1_'+ filename +'.csv')
history_cycle1 = pd.read_csv(path+'/History1_'+ filename +'.csv')

In [ ]:
#-- fit the model
print('-- beginning fit')
t1 = time.time()
history_ = model.fit( x_train, y_train, BATCH_SIZE, epochs=EPOCHS,\
                        validation_data=(x_test, y_test), verbose=True, callbacks=[reduce_lr])
final_accuracy = np.round(history_.history['val_loss'][-1],2)
print('-- fit complete')
print('-- time taken=', time.time() - t1)
print('-- final accuracy=', final_accuracy)

model.save(path+'/MODEL_'+filename, overwrite=True)
pd.DataFrame(history_.history).to_csv(path+'/History2_'+ filename +'.csv')

-- beginning fit
Epoch 1/100
266/266 [==============================] - 3s 10ms/step - loss: 642.2395 - val_loss: 638.0117 - lr: 3.1250e-05
Epoch 2/100
266/266 [==============================] - 2s 9ms/step - loss: 649.4267 - val_loss: 734.8665 - lr: 3.1250e-05
Epoch 3/100
266/266 [==============================] - 3s 10ms/step - loss: 672.2252 - val_loss: 770.4940 - lr: 3.1250e-05
Epoch 4/100
266/266 [==============================] - 3s 11ms/step - loss: 634.4815 - val_loss: 630.8138 - lr: 3.1250e-05
Epoch 5/100
266/266 [==============================] - 3s 10ms/step - loss: 638.2119 - val_loss: 806.5927 - lr: 3.1250e-05
Epoch 6/100
266/266 [==============================] - 3s 10ms/step - loss: 666.4365 - val_loss: 733.2562 - lr: 3.1250e-05
Epoch 7/100
266/266 [==============================] - 3s 10ms/step - loss: 656.2122 - val_loss: 588.3377 - lr: 3.1250e-05
Epoch 8/100
266/266 [==============================] - 3s 11ms/step - loss: 656.0500 - val_loss: 631.9960 - lr: 3.1250e-05


In [ ]:
history_cycle2 = pd.read_csv(path+'/History2_'+ filename +'.csv')

In [ ]:
history_stored=np.concatenate((history_cycle1['val_loss'],history_cycle2['val_loss']))

In [ ]:
fig = go.Figure()
fig.add_scatter(x=np.arange( 1, 200+1 ),y=history_stored ,line=dict(width=2,color='royalblue'),opacity=1,name=f'Validation loss')
fig.update_layout(template=fig_template,
                  xaxis = dict(title='Epochs'),
                  yaxis = dict(title='Mean absolute error on validation set',type="log",),
                  )
fig

## Testing the model

In [ ]:
loaded_model = models.load_model(path+'/MODEL_'+filename)
# loaded_model = models.load_model(path+'/MODEL_6.46_fil=20_ker=20_dr0.05_ps=2_LRini=0.001_LRmin=1e-06_bs=64_ep=500_NOISE_TYPES=lor')

In [102]:
#-- apply the network to the whole test set
predictions = loaded_model.predict(x_test)
predictions = model2d.predict(x_test)

#-- select random samples from the test data for illustration
# rand_set = np.random.randint( 0, y_test.shape[0] ,(8,) )

# c_predict = getCoherence(predictions,data_1['omega'],data_1['time'],1,100e-9)
# n_plot = 10
# rand_set = np.random.randint(0,predictions.shape[0],(n_plot,))

UnknownError: ignored

In [ ]:
fig = go.Figure()
for i in range(n_plot):
  fig.add_scatter(x=data_1['time']*1e6,y=x_test[rand_set[i]],line=dict(width=2,color=seq_colors[i]),opacity=0.8,name=f'{rand_set[i]}')
  fig.add_scatter(x=data_1['time']*1e6,y=c_predict[rand_set[i]],line=dict(width=3,color=seq_colors[i]),opacity=1,name=f'{rand_set[i]}')

fig.update_layout(template=fig_template)
fig.layout.xaxis.title = 'Total time (\N{greek small letter mu}s)'
fig.layout.yaxis.title = 'Coherence'
fig.update_xaxes(range=[0,400])
fig

In [ ]:
fig = go.Figure()
for i in range(n_plot):
  fig.add_scatter(x=data_1['omega']*1e-3,y=y_test[rand_set[i]],line=dict(width=2,color=seq_colors[i]),opacity=0.8,name=f'{rand_set[i]}')
  fig.add_scatter(x=data_1['omega']*1e-3,y=predictions[rand_set[i]],line=dict(width=3,color=seq_colors[i]),opacity=1,name=f'{rand_set[i]}')

fig.update_layout(template=fig_template)
fig.layout.xaxis.title = '\N{greek small letter omega} (kHz)'
fig.layout.yaxis.title = 'Noise Spectrum'
fig.update_yaxes(type="log")
fig.update_xaxes(range=[0,150])
fig.update_yaxes(range=[2,6])
fig

In [ ]:
def percent_error(data_test,data_predict):
  return np.mean(np.abs(data_test-data_predict)*100,axis=1)/np.mean(np.abs(data_test))

In [ ]:
err_perc = percent_error(y_test,predictions)

In [ ]:
np.mean(err_perc)

95.45482934517048

In [ ]:
hist_dense,x =np.histogram(err_perc,bins=51,range=[0,200],density=False)
x = (x[:-1]+x[1:])/2

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Training History','Mean Absolute Error'))

fig.add_scatter(x=np.arange(1,200+1),y=history_stored,line=dict(width=3,color='royalblue'),opacity=1,name=f'Validation loss',row=1,col=1,showlegend=False)
fig.add_trace(go.Bar(x=x,y=hist_dense,opacity=0.75,name='Error',showlegend=False,marker=dict(color='royalblue')),row=1,col=2)

fig.update_layout(template=fig_template,
                  width = 1000,
                  xaxis1 = dict(title='Epochs'),
                  yaxis1 = dict(title='Mean absolute error on validation set',type="log",),
                  xaxis2 = dict(title='% Error',range=[0,200]),
                  yaxis2 = dict(title='Samples'),
                  # xaxis3 = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,200]),
                  # yaxis3 = dict(title='Coherence'),
                 )

In [ ]:
# Plot
n_plot = 2100
fig = make_subplots(rows=1, cols=3, subplot_titles=('Velocity-Amplitude', 'Force-Amplitude-Calculated', 'Force-Amplitude-Predicted'))

fig.add_heatmap(x=np.arange(0,15),y=np.arange(0,15),z=np.squeeze(x_test.reshape((3000,15,15))[n_plot,:,:]), row=1, col=1,
                # zmin=0, zmax=1.1,
                showscale=False,
                # zsmooth='best',
                colorscale='Viridis',
                )

fig.add_heatmap(x=np.arange(0,15),y=np.arange(0,15),z=np.squeeze(y_test.reshape((3000,15,15))[n_plot,:,:]), row=1, col=2,
                # zmin=0, zmax=1e-3,
                showscale=False,
                # zsmooth='best',
                colorscale='Viridis',
                )
fig.add_heatmap(x=np.arange(0,15),y=np.arange(0,15),z=np.squeeze(predictions.reshape((3000,15,15))[n_plot,:,:]), row=1, col=3,
                # zmin=0, zmax=1e-3,
                showscale=True,
                # zsmooth='best',
                colorscale='Viridis',
                )


fig.layout.xaxis1.title = 'X'
fig.layout.yaxis1.title = 'Y'
fig.layout.xaxis2.title = 'X'
fig.layout.yaxis2.title = 'Y'

fig.update_layout(height=525,width=1000)
fig

In [ ]:
predictions.shape

(3000, 225)

In [ ]:
percent_error(y_test[n_plot,:].reshape((1,225)),predictions[n_plot,:].reshape((1,225)))

array([1.02607092e+30])

In [ ]:
y_test[n_plot,:].shape

(225,)

In [ ]:
random_input = np.random.random_sample((2,225))

In [ ]:
random_output = loaded_model.predict(random_input)

1/1 [==============================] - 0s 22ms/step


In [ ]:
fig = go.Figure()
fig.add_heatmap(x=np.arange(0,15),y=np.arange(0,15),z=np.squeeze(random_output.reshape((2,15,15))[0,:,:]),
                # zmin=0, zmax=1e-3,
                showscale=False,
                # zsmooth='best',
                colorscale='Viridis',
                )

fig.update_layout(template=fig_template,
                  xaxis = dict(title='X'),
                  yaxis = dict(title='Y'),
                  height = 600, width = 600,
                  )
fig